In [50]:
import tensorflow as tf

In [51]:
my_tensor = tf.constant(5)  #Constant
my_variable = tf.Variable([[4,6],[5,6]])  #Variable

#Variable can also be of different types as simple Tensor

bool_variable = tf.Variable([True, False])  #Boolean Variable
complex_variable = tf.Variable([6 + 5j, 4 - 7j])  #Complex Variable! Complex? Yeah They Also Exist in Python lol

In [52]:
print("Shape ", my_variable.shape)
print("Dtype ", my_variable.dtype)
print("As Numpy\n", my_variable.numpy())

Shape  (2, 2)
Dtype  <dtype: 'int32'>
As Numpy
 [[4 6]
 [5 6]]


In [53]:
# Most Tensor Operations Can Be Applied On Variables Although Variables Can Not Be Reshaped

print("A Variable\n", my_variable)
print("\nViewed As A Tensor\n", tf.convert_to_tensor(my_variable))
print("\nIndex Of Highest Value ", tf.argmax(my_variable))

# This Creates a New tensor it doesnot reshape the Original One

print("\nCopying and Reshaped\n",tf.reshape(my_variable, [1,4]))

A Variable
 <tf.Variable 'Variable:0' shape=(2, 2) dtype=int32, numpy=
array([[4, 6],
       [5, 6]], dtype=int32)>

Viewed As A Tensor
 tf.Tensor(
[[4 6]
 [5 6]], shape=(2, 2), dtype=int32)

Index Of Highest Value  tf.Tensor([1 0], shape=(2,), dtype=int64)

Copying and Reshaped
 tf.Tensor([[4 6 5 6]], shape=(1, 4), dtype=int32)


In [54]:
from re import template
a = tf.Variable([4.0, 5.0])
a.assign([1,2]) #This will assign the values to existing values and would keep the dtype as original
print(a, "\n")

try:
  a.assign([1.0, 2.0, 3.0])

except Exception as e:
  print(e) #Because it would try to reshape the Variable and reshaping means to change the Size of Variable 

<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([1., 2.], dtype=float32)> 

Cannot assign value to variable ' Variable:0': Shape mismatch.The variable shape (2,), and the assigned value shape (3,) are incompatible.


In [55]:
a = tf.Variable([2.0, 3.0])
# Create b based on the value of a

b = tf.Variable(a)
a.assign([5,6])

# a and b are different

print(a.numpy(),"\n")
print(b.numpy(),"\n")

# There are other versions of assign

print(a.assign_add([2,3]).numpy(),"\n")
print(a.assign_sub([7,9]).numpy(),"\n")

[5. 6.] 

[2. 3.] 

[7. 9.] 

[0. 0.] 



In [56]:
# Creating a and b They Have the same value but is backed by different tensors
a = tf.Variable(my_variable, name= "Mark")

# A New Variable with same name but different Value
b= tf.Variable(my_variable + 1, name= "Mark") #Names of Variables are assigned and preserved automatically so no worries unless you want to 

print(a == b)


tf.Tensor(
[[False False]
 [False False]], shape=(2, 2), dtype=bool)


In [57]:
step_count = tf.Variable(1, trainable = False)
# If True, GradientTapes automatically watch uses of this variable.
# Defaults to True, unless synchronization is set to ON_READ, in which case it defaults to False.

In [58]:
# TensorFlow tries to place the Varibales on fastest hardware thats why it Places the Variable on GPUS i available 
# but we can override this snippet as well

with tf.device("CPU:0"):
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0 ,5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

  c = tf.matmul(a, b)

print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [59]:
with tf.device("CPU:0"):
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.Variable([[1.0, 2.0, 3.0]])

with tf.device("GPU:0"):
  k = a*b

print(k)

tf.Tensor(
[[ 1.  4.  9.]
 [ 4. 10. 18.]], shape=(2, 3), dtype=float32)


# **Writing Low Level API**

# Variable VS Constant
Tensors in TensorFlow are either contant (tf.constant) or variables (tf.Variable). Constant values can not be changed, while variables values can be.

The main difference is that instances of tf.Variable have methods allowing us to change their values while tensors constructed with tf.constant don't have these methods, and therefore their values can not be changed. When you want to change the value of a tf.Variable x use one of the following method:

x.assign(new_value)
x.assign_add(value_to_be_added)
x.assign_sub(value_to_be_subtracted


In [60]:
import numpy as np
import matplotlib.pyplot as plt

In [61]:
X = tf.constant(range(10), dtype = "float32")
Y = 2 * X + 10

In [62]:
print("X: ", X)
print("Y: ", Y)

X:  tf.Tensor([0. 1. 2. 3. 4. 5. 6. 7. 8. 9.], shape=(10,), dtype=float32)
Y:  tf.Tensor([10. 12. 14. 16. 18. 20. 22. 24. 26. 28.], shape=(10,), dtype=float32)


In [105]:
X_test = tf.constant(range(10, 20), dtype = "float32")
Y_test = 2 * X_test + 10

In [106]:
print("X_test: ", X_test)
print("Y_test: ", Y_test)

X_test:  tf.Tensor([10. 11. 12. 13. 14. 15. 16. 17. 18. 19.], shape=(10,), dtype=float32)
Y_test:  tf.Tensor([30. 32. 34. 36. 38. 40. 42. 44. 46. 48.], shape=(10,), dtype=float32)


In [107]:
y_mean = Y.numpy().mean()

def predict_mean(x):
  y_hat = [y_mean] * len(x)
  return y_hat

Y_hat = predict_mean(X_test)

errors = (Y_hat - Y)**2
loss = tf.reduce_mean(errors)
loss.numpy()

33.0

In [108]:
def loss_mse(X, Y, w0, w1):
  Y_hat = (w0 * X) + w1
  error = (Y_hat - Y)**2
  return tf.reduce_mean(error)  #Reduce Mean is basically the basic mean

In [109]:
def compute_gradient(X, Y , w0, w1):
  with tf.GradientTape() as tape:
    loss = loss_mse(X, Y, w0, w1)
  return tape.gradient(loss, [w0, w1]) # Returns Two Values one for w0, and one for w1

In [110]:
w0 = tf.Variable(0.0)
w1 = tf.Variable(0.0)

dw0, dw1 = compute_gradient(X, Y, w0, w1) 

In [111]:
print("dw0: ", dw0.numpy())
print("dw1: ", dw1.numpy())

dw0:  -204.0
dw1:  -38.0


In [114]:
steps = 1000
learning_rate = 0.02
MSG = "Step: {step}, loss: {loss}, w0: {w0}, w1: {w1}"


w0 = tf.Variable(0.0)
w1 = tf.Variable(0.0)



for step in range(0, steps + 1):
  
  dw0 , dw1 = compute_gradient(X, Y, w0, w1)
  w0.assign_sub(dw0 * learning_rate)
  w1.assign_sub(dw1 * learning_rate)

  if step % 100 == 0:
    loss = loss_mse(X, Y, w0, w1)
    print(MSG.format(step = step, loss = loss, w0 = w0.numpy(), w1 = w1.numpy()))


Step: 0, loss: 35.70719528198242, w0: 4.079999923706055, w1: 0.7599999904632568
Step: 100, loss: 2.6017532348632812, w0: 2.4780430793762207, w1: 7.002389907836914
Step: 200, loss: 0.26831889152526855, w0: 2.153517961502075, w1: 9.037351608276367
Step: 300, loss: 0.027671903371810913, w0: 2.0493006706237793, w1: 9.690855979919434
Step: 400, loss: 0.0028539239428937435, w0: 2.0158326625823975, w1: 9.90071964263916
Step: 500, loss: 0.0002943490108009428, w0: 2.005084753036499, w1: 9.96811580657959
Step: 600, loss: 3.0356444767676294e-05, w0: 2.0016329288482666, w1: 9.989760398864746
Step: 700, loss: 3.1322738323069643e-06, w0: 2.0005245208740234, w1: 9.996710777282715
Step: 800, loss: 3.2238213520940917e-07, w0: 2.0001683235168457, w1: 9.998944282531738
Step: 900, loss: 3.369950718479231e-08, w0: 2.000054359436035, w1: 9.999658584594727
Step: 1000, loss: 3.6101481803996194e-09, w0: 2.0000178813934326, w1: 9.99988842010498


In [116]:
loss = loss_mse(X_test, Y_test, w0, w1)
loss.numpy()

2.4563633e-08